In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
from torchtext import data
import json
import os
from torchtext import data
from collections import Counter
import torchtext
import pickle

tqdm.pandas()

In [ ]:
train_data = torch.load('/content/drive/My Drive/IRE_project/train_data_bypublisher.dataset')
val_data = torch.load('/content/drive/My Drive/IRE_project/val_data_bypublisher.dataset')
byarticle_data = torch.load('/content/drive/My Drive/IRE_project/train_data_byarticle.dataset')
f = open('/content/drive/My Drive/IRE_project/dict1.json')
dict1 = json.load(f)

In [ ]:
my_dict = Counter(dict1)

In [ ]:
# vocab = torchtext.vocab.Vocab(my_dict, vectors='glove.6B.100d')

In [ ]:
#loaded data in batches
batch_size = 256

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size, drop_last=True)
byarticle_loader = DataLoader(byarticle_data, shuffle=True, batch_size = 16, drop_last = True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
train_on_gpu = False
if torch.cuda.is_available():
  train_on_gpu = True
print(device)
print(train_on_gpu)

cuda
True


In [ ]:
def f1_score_func(preds, labels):
    # preds_flat = np.argmax(preds, axis=1).flatten()
    preds_flat = preds.flatten()
    labels_flat = labels.flatten()
    # scores_flat = scores.flatten()
#     print(preds.shape)i think
#     print(labels.shape)
    log_loss = nn.BCELoss()
    # coef, p = spearmanr(scores_flat, preds_flat)
    return log_loss(torch.tensor(preds).float(), torch.tensor(labels)), f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    # label_dict_inverse = {v: k for k, v in label_dict.items()}
    label_dict_inverse = {1: 'Hyperpartisan', 0: 'Not hyperpartisan'}
    preds_flat = preds.flatten()
    labels_flat = labels.flatten()
    l_preds=0
    l_true=0
    accuracies = []
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        l_preds += len(y_preds[y_preds==label])
        l_true += len(y_true)
        print(f'Class: {label_dict_inverse[label]}')
        accuracies.append(f'{len(y_preds[y_preds==label])}/{len(y_true)}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} \t {len(y_preds[y_preds==label])/len(y_true)}\n')
    print(f'Total Accuracy: {l_preds/l_true}')
    return accuracies, l_preds/l_true

In [ ]:
class SentimentRNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5, vocab=None, m_type='LSTM'):
        super(SentimentRNN, self).__init__()
        
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.emb_type = 'default'
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.model = m_type
        #embedding and LSTM layers
        if vocab != None:
          print('using glove')
          self.embedding.weight.data.copy_(vocab)
          self.embedding.weight.requires_grad = False
          # print('hello')
        # else:
        #   print('using glove')
        #   self.emb_type = 'glove'
        #   self.embedding = vocab
        if m_type == 'LSTM':
          self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout = drop_prob, batch_first = True)
        else:
          self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, dropout = drop_prob, batch_first = True)
        
        self.dropout = nn.Dropout(0.2)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    def forward(self, x, hidden):
        
        batch_size = x.size(0)
        
        x = x.long()
        # if self.emb_type == 'default':
        embeds = self.embedding(x)
        # else:
        #   embeds = self.embedding[x].reshape(x.size()[0], -1)
        #   embeds = embeds.reshape(-1, 512, 100)
        # embeds.to(device)
        lstm_out, hidden = self.lstm(embeds, hidden)
        #stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
#         print(out.shape)
        sig_out = self.sig(out)
        
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] #getting the batch of labels
        #return the last sigmoid output and hidden state
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if train_on_gpu:
            if self.model == 'LSTM':
              hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
            else:
              hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda()
        else:
            if self.model == 'LSTM':
              hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
            else:
              hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        
        return hidden

In [ ]:
vocab_size = len(dict1)+1
output_size = 1
embedding_dim = 100
hidden_dim = 256
n_layers = 2
model_type = 'GRU'
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, m_type=model_type)
# net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, vocab=vocab.vectors)
net.to(device)
print(net)
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(total_params)

SentimentRNN(
  (embedding): Embedding(1995845, 100)
  (lstm): GRU(100, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)
200254453


In [ ]:
lr = 0.001
epochs = 10
counter = 0
print_every = 100
clip = 5

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
def evaluate(val_loader, bs):
  val_h = net.init_hidden(bs)
  val_losses = []
  net.eval()
  num_correct = 0
  # with torch.no_grad():
  cnt = 0
  val_preds = []
  true_vals = []
  loss_val_total = 0
  # for inputs, labels in val_loader:
  prev_out = torch.zeros(256)
  for batch in tqdm(val_loader):
      batch = tuple(b for b in batch)
      inputs, labels = batch[0], batch[1]
      if train_on_gpu:
          inputs, labels = inputs.to(device), labels.to(device)
      if model_type == 'GRU':
        val_h = val_h.data
      else:
        val_h = tuple([each.data for each in val_h])
      # net.zero_grad()
      # try:t
      output, val_h = net(inputs, val_h)
      # except:
      #   output = prev_out
      val_loss = criterion(output.squeeze(), labels.float())
      loss_val_total += val_loss.item()
      val_losses.append(val_loss.item())
      prev_out = output
      pred = torch.round(output.squeeze())
      val_preds.append(pred.cpu().data.numpy())
      true_vals.append(labels.float().cpu().data.numpy())
      correct_tensor = pred.eq(labels.float().view_as(pred))
      correct = np.squeeze(correct_tensor.cpu().numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
      num_correct += np.sum(correct)
      # torch.cuda.empty_cache()
  # net.train()
  loss_val_avg = loss_val_total/len(val_loader)
  val_preds = np.array(val_preds)
  true_vals = np.array(true_vals)
  return loss_val_avg, val_preds, true_vals, num_correct

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
output_list = []
for e in tqdm(range(1, epochs+1)):
    net.train()
#     # initialize hidden state
    h = net.init_hidden(batch_size)
    num_correct_train = 0
    # batch loop
    # progress_bar = tqdm(val_loader, desc='Epoch {:1d}'.format(e), leave=False, disable=False)
    progress_bar = tqdm(train_loader, desc='Epoch {:1d}'.format(e), leave=False, disable=False)

    for batch in progress_bar:
        counter += 1
        batch = tuple(b for b in batch)
        inputs, labels = batch[0], batch[1]
        if train_on_gpu:
            inputs, labels = inputs.to(device), labels.to(device)

#         # Creating new variables for the hidden state, otherwise
#         # we'd backprop through the entire training history
        if model_type == 'GRU':
          h = h.data
        else:
          h = tuple([each.data for each in h])
#         # else:
#         #     h = h.data
        # print(type(h))
#         # zero accumulated gradients
        net.zero_grad()
        # h.to(device)
#         # get the output from the model
        output, h = net(inputs, h)
# #         print(output.shape)
# #         print(labels.shape)
# # #         break
#         # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
#         # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        pred = torch.round(output.squeeze())
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

        correct_tensor = pred.eq(labels.float().view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        num_correct_train += np.sum(correct)
        torch.cuda.empty_cache()
    train_acc = num_correct_train/len(train_loader.dataset)
    torch.save(net.state_dict(), f'bypublisher_lstm_{embedding_dim}_epoch_{e}.model')
    loss_val, val_preds, true_vals, num_correct = evaluate(val_loader, batch_size)
    _, f1 = f1_score_func(val_preds, true_vals)
    loss_byart, art_preds, art_true, art_num_correct = evaluate(byarticle_loader, 16)
    _, f1_art = f1_score_func(art_preds, art_true)
    print("Epoch: {}/{}...".format(e, epochs),
          "Step: {}...".format(counter),
          "Loss: {:.6f}...".format(loss.item()),
          "Train Accuracy: {:.3f}".format(train_acc),
          "Val Loss: {:.6f}".format(loss_val),
          "Val Accuracy: {:.3f}".format(num_correct/len(val_loader.dataset)),
          "Val F1-Score: {:.3f}".format(f1),
          "Byarticle Loss: {:.6f}".format(loss_byart),
          "Byarticle Accuracy: {:.3f}".format(art_num_correct/len(byarticle_loader.dataset)),
          "Byarticle F1-Score: {:.3f}".format(f1_art))
    accuracies, total_acc = accuracy_per_class(val_preds, true_vals)
    art_accs, art_total_acc = accuracy_per_class(art_preds, art_true)
    output_list.append({'epoch': e, 'loss': loss.item(), 'train_acc': train_acc, 'val_loss': loss_val, 'val_acc': total_acc,
                        'val_accuracies': str(accuracies), 'byarticle_loss': loss_byart, 'byarticle_acc': art_total_acc,
                        'byarticle_accuracies': str(art_accs), 'val_f1': f1, 'byarticle_f1': f1_art})


Epoch: 1/10... Step: 2343... Loss: 0.161293... Train Accuracy: 0.885 Val Loss: 1.191519 Val Accuracy: 0.562 Val F1-Score: 0.533 Byarticle Loss: 1.544326 Byarticle Accuracy: 0.364 Byarticle F1-Score: 0.197
Class: Not hyperpartisan
Accuracy: 23421/74876 	 0.3127971579678402

Class: Hyperpartisan
Accuracy: 60810/74884 	 0.8120559799156027

Total Accuracy: 0.5624399038461538
Class: Not hyperpartisan
Accuracy: 0/405 	 0.0

Class: Hyperpartisan
Accuracy: 235/235 	 1.0

Total Accuracy: 0.3671875



Epoch: 2/10... Step: 4686... Loss: 0.143772... Train Accuracy: 0.949 Val Loss: 1.157348 Val Accuracy: 0.567 Val F1-Score: 0.543 Byarticle Loss: 1.510440 Byarticle Accuracy: 0.367 Byarticle F1-Score: 0.200
Class: Not hyperpartisan
Accuracy: 24941/74874 	 0.33310628522584607

Class: Hyperpartisan
Accuracy: 60175/74886 	 0.8035547365328632

Total Accuracy: 0.568349358974359
Class: Not hyperpartisan
Accuracy: 0/403 	 0.0

Class: Hyperpartisan
Accuracy: 237/237 	 1.0

Total Accuracy: 0.3703125



Epoch: 3/10... Step: 7029... Loss: 0.070034... Train Accuracy: 0.966 Val Loss: 1.424118 Val Accuracy: 0.553 Val F1-Score: 0.521 Byarticle Loss: 1.893739 Byarticle Accuracy: 0.363 Byarticle F1-Score: 0.196
Class: Not hyperpartisan
Accuracy: 21905/74881 	 0.2925308155606896

Class: Hyperpartisan
Accuracy: 60981/74879 	 0.8143938888072757

Total Accuracy: 0.5534588675213675
Class: Not hyperpartisan
Accuracy: 0/406 	 0.0

Class: Hyperpartisan
Accuracy: 234/234 	 1.0

Total Accuracy: 0.365625



Epoch: 4/10... Step: 9372... Loss: 0.052733... Train Accuracy: 0.979 Val Loss: 1.544762 Val Accuracy: 0.564 Val F1-Score: 0.552 Byarticle Loss: 1.988262 Byarticle Accuracy: 0.361 Byarticle F1-Score: 0.194
Class: Not hyperpartisan
Accuracy: 29432/74884 	 0.3930345601196517

Class: Hyperpartisan
Accuracy: 55212/74876 	 0.7373791335007212

Total Accuracy: 0.5651976495726496
Class: Not hyperpartisan
Accuracy: 0/407 	 0.0

Class: Hyperpartisan
Accuracy: 233/233 	 1.0

Total Accuracy: 0.3640625



Epoch: 5/10... Step: 11715... Loss: 0.034563... Train Accuracy: 0.987 Val Loss: 1.914975 Val Accuracy: 0.554 Val F1-Score: 0.532 Byarticle Loss: 2.282072 Byarticle Accuracy: 0.366 Byarticle F1-Score: 0.199
Class: Not hyperpartisan
Accuracy: 24738/74883 	 0.33035535435279034

Class: Hyperpartisan
Accuracy: 58431/74877 	 0.7803597900556913

Total Accuracy: 0.5553485576923077
Class: Not hyperpartisan
Accuracy: 0/404 	 0.0

Class: Hyperpartisan
Accuracy: 236/236 	 1.0

Total Accuracy: 0.36875



Epoch: 6/10... Step: 14058... Loss: 0.007439... Train Accuracy: 0.992 Val Loss: 2.283247 Val Accuracy: 0.565 Val F1-Score: 0.552 Byarticle Loss: 1.931176 Byarticle Accuracy: 0.364 Byarticle F1-Score: 0.197
Class: Not hyperpartisan
Accuracy: 29070/74882 	 0.3882107849683502

Class: Hyperpartisan
Accuracy: 55647/74878 	 0.7431688880579075

Total Accuracy: 0.5656850961538461
Class: Not hyperpartisan
Accuracy: 0/405 	 0.0

Class: Hyperpartisan
Accuracy: 235/235 	 1.0

Total Accuracy: 0.3671875



Epoch: 7/10... Step: 16401... Loss: 0.020019... Train Accuracy: 0.994 Val Loss: 2.660787 Val Accuracy: 0.549 Val F1-Score: 0.522 Byarticle Loss: 2.485429 Byarticle Accuracy: 0.364 Byarticle F1-Score: 0.197
Class: Not hyperpartisan
Accuracy: 23034/74879 	 0.30761628761067855

Class: Hyperpartisan
Accuracy: 59269/74881 	 0.7915091945887475

Total Accuracy: 0.5495659722222223
Class: Not hyperpartisan
Accuracy: 0/405 	 0.0

Class: Hyperpartisan
Accuracy: 235/235 	 1.0

Total Accuracy: 0.3671875



Epoch: 8/10... Step: 18744... Loss: 0.050187... Train Accuracy: 0.995 Val Loss: 2.668788 Val Accuracy: 0.562 Val F1-Score: 0.546 Byarticle Loss: 2.950080 Byarticle Accuracy: 0.366 Byarticle F1-Score: 0.199
Class: Not hyperpartisan
Accuracy: 27564/74870 	 0.3681581407773474

Class: Hyperpartisan
Accuracy: 56746/74890 	 0.7577246628388303

Total Accuracy: 0.5629674145299145
Class: Not hyperpartisan
Accuracy: 0/404 	 0.0

Class: Hyperpartisan
Accuracy: 236/236 	 1.0

Total Accuracy: 0.36875



Epoch: 9/10... Step: 21087... Loss: 0.016685... Train Accuracy: 0.995 Val Loss: 3.029030 Val Accuracy: 0.552 Val F1-Score: 0.527 Byarticle Loss: 0.737766 Byarticle Accuracy: 0.366 Byarticle F1-Score: 0.199
Class: Not hyperpartisan
Accuracy: 23832/74887 	 0.31823948081776543

Class: Hyperpartisan
Accuracy: 58954/74873 	 0.7873866413794025

Total Accuracy: 0.5527911324786324
Class: Not hyperpartisan
Accuracy: 0/404 	 0.0

Class: Hyperpartisan
Accuracy: 236/236 	 1.0

Total Accuracy: 0.36875



Epoch: 10/10... Step: 23430... Loss: 0.004194... Train Accuracy: 0.996 Val Loss: 3.033231 Val Accuracy: 0.549 Val F1-Score: 0.522 Byarticle Loss: 2.611989 Byarticle Accuracy: 0.366 Byarticle F1-Score: 0.199
Class: Not hyperpartisan
Accuracy: 23230/74888 	 0.3101965601965602

Class: Hyperpartisan
Accuracy: 59125/74872 	 0.7896810556683407

Total Accuracy: 0.5499131944444444
Class: Not hyperpartisan
Accuracy: 0/404 	 0.0

Class: Hyperpartisan
Accuracy: 236/236 	 1.0

Total Accuracy: 0.36875



In [ ]:
json_data = json.dumps(output_list, indent=2)
with open('gru_'+str(embedding_dim)+'_'+str(batch_size)+'.json', 'w') as out:
  out.write(json_data)
out.close()

In [ ]:
out_df = pd.DataFrame(output_list)
out_df.head()

,epoch,loss,train_acc,val_loss,val_acc,val_accuracies,byarticle_loss,byarticle_acc,byarticle_accuracies,val_f1,byarticle_f1
0,1,0.161293,0.884547,1.191519,0.562440,"['23421/74876', '60810/74884']",1.544326,0.367188,"['0/405', '235/235']",0.533360,0.197232
1,2,0.143772,0.949445,1.157348,0.568349,"['24941/74874', '60175/74886']",1.510440,0.370312,"['0/403', '237/237']",0.543065,0.200146
2,3,0.070034,0.966423,1.424118,0.553459,"['21905/74881', '60981/74879']",1.893739,0.365625,"['0/406', '234/234']",0.520835,0.195781
3,4,0.052733,0.978560,1.544762,0.565198,"['29432/74884', '55212/74876']",1.988262,0.364063,"['0/407', '233/233']",0.551916,0.194333
4,5,0.034563,0.987013,1.914975,0.555349,"['24738/74883', '58431/74877']",2.282072,0.368750,"['0/404', '236/236']",0.531638,0.198687


In [ ]:
out_df.to_csv('gru_'+str(embedding_dim)+'_'+str(batch_size)+'.csv', index=False)

In [ ]:
! cp *.csv /content/drive/My\ Drive/IRE_project
! cp *.json /content/drive/My\ Drive/IRE_project

In [ ]:
for i in range(1, 7):
  net.load_state_dict(torch.load(f'bypublisher_lstm_'+str(embedding_dim)+'_epoch_'+str(i)+'.model', map_location=device))
  net.eval()
  loss_byart, art_preds, art_true, art_num_correct = evaluate(byarticle_loader, 16)
  _, f1_art = f1_score_func(art_preds, art_true)
  art_accs, art_total_acc = accuracy_per_class(art_preds, art_true)
  print(f"Epoch: {i}\tF1-Score: {f1_art}, Loss: {loss_byart}")

In [ ]:
! cp -r bypublisher_gru_100_epoch_2.model /content/drive/My\ Drive

In [ ]:
! du -csh *.model

764M	bypublisher_gru_100_epoch_2.model
764M	bypublisher_lstm_100_epoch_10.model
764M	bypublisher_lstm_100_epoch_1.model
764M	bypublisher_lstm_100_epoch_2.model
764M	bypublisher_lstm_100_epoch_3.model
764M	bypublisher_lstm_100_epoch_4.model
764M	bypublisher_lstm_100_epoch_5.model
764M	bypublisher_lstm_100_epoch_6.model
764M	bypublisher_lstm_100_epoch_7.model
764M	bypublisher_lstm_100_epoch_8.model
764M	bypublisher_lstm_100_epoch_9.model
8.3G	total
